In [84]:
import pandas as pd
import numpy as np
df = pd.read_csv('vxx2020.csv')
# housekeeping
df.drop(['symbol'], inplace=True, axis=1)
# add - to date, exdate, and last date after 4 characters
df['date'] = pd.to_datetime(df['date'].astype(str), format='%Y%m%d')
df['exdate'] = pd.to_datetime(df['exdate'].astype(str), format='%Y%m%d')
df['last_date'] = pd.to_datetime(df['last_date'].astype(str), format='%Y%m%d')
df.head()

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid
0,2020-01-02,2020-01-03,2019-12-17,C,10500,3.90,4.10,0,0,NaN,NaN,NaN,NaN,NaN,131456095
1,2020-01-02,2020-01-03,2019-12-30,C,11000,3.40,3.60,0,0,NaN,NaN,NaN,NaN,NaN,131456096
2,2020-01-02,2020-01-03,2019-12-30,C,11500,3.00,3.05,0,9,2.267680,0.978255,0.030210,0.039436,-16.53450,131456097
3,2020-01-02,2020-01-03,2020-01-02,C,12000,2.52,2.53,30,274,1.908156,0.974564,0.040960,0.045100,-15.88719,131456098
4,2020-01-02,2020-01-03,2020-01-02,C,12500,2.01,2.03,9,390,1.448381,0.977563,0.048564,0.040484,-10.92112,131456099


In [85]:
# add price for VXX
from functions import import_data, check_contango
today = pd.to_datetime('today').strftime('%Y-%m-%d')

tickers = ['^VIX','^VIX3M', 'VXX']

VIX = import_data(tickers[0], '2010-01-01', today, '1d').reset_index()
VIX3M = import_data(tickers[1], '2010-01-01', today, '1d').reset_index()
VXX = import_data(tickers[2], '2020-01-01', today, '1d').reset_index()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [87]:
print(df.cp_flag.value_counts())
print('Interesting, there are two more puts than calls total.')

P    203278
C    203276
Name: cp_flag, dtype: int64
Interesting, there are two more puts than calls total.


In [88]:
# drop all rows with 0 volume
print(len(df))
df = df[df['volume'] != 0]
print(len(df))
df.head(20)
print(df.cp_flag.value_counts())


406554
233018
C    137032
P     95986
Name: cp_flag, dtype: int64


After removing all options with zero volume, we have about 30k more active call options than puts.  
Interesting!

In [101]:
# Add VXX, VIX, VIX3M prices to df at the correct date.
# Create empto columns for VXX, VIX, VIX3M
df['VXX'] = np.nan
df['VIX'] = np.nan
df['VIX3M'] = np.nan

# This is going to be slow.
for i, v in df.iterrows():
    df['VXX'].loc[i] = VXX['Close'].where(VXX['Date'] == v['date'])

<ipython-input-101-6267478eddbe>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['VXX'].loc[i] = VXX['Close'].where(VXX['Date'] == v['date'])


KeyboardInterrupt: 

In [94]:
df

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid
3,2020-01-02,2020-01-03,2020-01-02,C,12000,2.52,2.53,30,274,1.908156,0.974564,0.040960,0.045100,-15.88719,131456098
4,2020-01-02,2020-01-03,2020-01-02,C,12500,2.01,2.03,9,390,1.448381,0.977563,0.048564,0.040484,-10.92112,131456099
5,2020-01-02,2020-01-03,2020-01-02,C,13000,1.51,1.53,8,389,1.118827,0.971736,0.076274,0.049281,-10.25413,131456100
6,2020-01-02,2020-01-03,2020-01-02,C,13500,1.02,1.04,130,366,0.916016,0.936959,0.178060,0.094039,-15.93107,131456101
7,2020-01-02,2020-01-03,2020-01-02,C,14000,0.53,0.55,833,3756,0.606814,0.873601,0.450219,0.157407,-17.64780,131456102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406467,2021-12-31,2022-01-21,2021-12-31,C,7000,0.07,0.12,2,4714,NaN,NaN,NaN,NaN,NaN,131209856
406483,2021-12-31,2022-01-21,2021-12-31,P,12000,7.35,7.45,5,18058,NaN,NaN,NaN,NaN,NaN,130543633
406489,2021-12-31,2022-01-21,2021-12-31,P,14000,9.35,9.45,5,2246,NaN,NaN,NaN,NaN,NaN,130543635
406499,2021-12-31,2022-01-21,2021-12-31,P,20000,15.30,15.45,50,1750,NaN,NaN,NaN,NaN,NaN,130543641


In [100]:
# search vxx for Date 2020-01-02
VXX[VXX['Date'] == '2020-01-02'].Close

0    58.040001
Name: Close, dtype: float64

In [102]:
VXX

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-01-02,58.840000,60.160000,58.000000,58.040001,58.040001,7078100
1,2020-01-03,62.680000,62.959999,59.799999,61.160000,61.160000,12039150
2,2020-01-06,62.959999,63.164001,60.320000,60.320000,60.320000,6514300
3,2020-01-07,60.639999,61.439999,59.400002,60.040001,60.040001,6833775
4,2020-01-08,59.639999,60.000000,57.320000,59.200001,59.200001,12063125
...,...,...,...,...,...,...,...
564,2022-03-29,25.100000,25.490000,24.219999,24.850000,24.850000,5172500
565,2022-03-30,25.280001,25.809999,24.780001,25.000000,25.000000,3986400
566,2022-03-31,25.330000,26.020000,25.209999,25.700001,25.700001,2966400
567,2022-04-01,25.570000,25.889999,25.000000,25.030001,25.030001,3069700


In [104]:
vxx_close = pd.DataFrame()
vxx_close = VXX[["Date", "Close"]]
df_combined = pd.DataFrame()
df_combined = df.merge(VXX['Close'], how = 'left', left_on = 'date', right_on = 'Date')
df_combined.head()

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,...,VXX,VIX,VIX3M,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-01-02,2020-01-03,2020-01-02,C,12000,2.52,2.53,30,274,1.908156,...,NaN,NaN,NaN,2020-01-02,58.84,60.16,58.0,58.040001,58.040001,7078100
1,2020-01-02,2020-01-03,2020-01-02,C,12500,2.01,2.03,9,390,1.448381,...,NaN,NaN,NaN,2020-01-02,58.84,60.16,58.0,58.040001,58.040001,7078100
2,2020-01-02,2020-01-03,2020-01-02,C,13000,1.51,1.53,8,389,1.118827,...,NaN,NaN,NaN,2020-01-02,58.84,60.16,58.0,58.040001,58.040001,7078100
3,2020-01-02,2020-01-03,2020-01-02,C,13500,1.02,1.04,130,366,0.916016,...,NaN,NaN,NaN,2020-01-02,58.84,60.16,58.0,58.040001,58.040001,7078100
4,2020-01-02,2020-01-03,2020-01-02,C,14000,0.53,0.55,833,3756,0.606814,...,NaN,NaN,NaN,2020-01-02,58.84,60.16,58.0,58.040001,58.040001,7078100
